In [1]:
import pandas as pd
import numpy as np
import json
from glob import glob
import os
import plotly.express as px
from tqdm import tqdm
import re

tf_logdir = '../tf_logs/'
weights_dir = '../weights/'

In [2]:
def get_finished_configs(tf_logdir):
    """
    Get the list of finished configs, return a list.
    """
    if not os.path.exists(tf_logdir):
        raise ValueError('The directory {} does not exist.'.format(tf_logdir))

    configs, dirs = [], []
    for dir in os.listdir(tf_logdir):
        if os.path.exists(os.path.join(tf_logdir, dir, 'config.json')):
            dirs.append(dir)
            with open(os.path.join(tf_logdir, dir, 'config.json')) as f:
                config = json.load(f)
                configs.append(config)
    return configs, dirs

def get_columns(configs, dirs): 

    # data-related columns
    datasets = []
    epochs = []
    batch_size = []
    batch_size_limit = []
    num_points = []

    # model-related columns
    version = []
    backbone = []
    pooling = []
    pointnet = []
    pointnet_pnt2s = []
    self_attention = []
    self_attention_num_layers = []
    pointnet_cross_attention = []
    pointnet_cross_attention_attention_types = []
    multi_cross_attention = []
    multi_cross_attention_attention_types = []

    # result columns
    loss = []
    average_recall = []
    average_1p_recall = []

    for config in configs:
        num_points.append(config['params']['num_points'])
        datasets.append(config['params']['dataset_name'])
        epochs.append(config['params']['epochs'])
        batch_size.append(config['params']['batch_size'])
        batch_size_limit.append(config['params']['batch_size_limit'])
        backbone.append(config['params']['model_params']['backbone'])
        pooling.append(config['params']['model_params']['pooling'])

        pointnet.append(True if 'pointnet' in config['params']['model_params']['combine_params'] else False)
        pointnet_pnt2s.append(config['params']['model_params']['combine_params']['pointnet']['pnt2s'] if \
                             'pointnet' in config['params']['model_params']['combine_params'] else None)
        self_attention.append(True if 'self_attention' in config['params']['model_params']['combine_params'] else False)
        self_attention_num_layer = 1 if 'self_attention' in config['params']['model_params']['combine_params'] else None
        self_attention_num_layer = config['params']['model_params']['combine_params']['self_attention']['num_layers'] if \
                                    'self_attention' in config['params']['model_params']['combine_params'] and \
                                    'num_layers' in config['params']['model_params']['combine_params']['self_attention'] else self_attention_num_layer
        self_attention_num_layers.append(self_attention_num_layer)
        pointnet_cross_attention.append(True if 'pointnet_cross_attention' in config['params']['model_params']['combine_params'] else False)
        pointnet_cross_attention_attention_types.append(config['params']['model_params']['combine_params']['pointnet_cross_attention']['attention_type'] if \
                                                        'pointnet_cross_attention' in config['params']['model_params']['combine_params'] else None)
        multi_cross_attention.append(True if 'multi_cross_attention' in config['params']['model_params']['combine_params'] else False)
        multi_cross_attention_attention_types.append(config['params']['model_params']['combine_params']['multi_cross_attention']['attention_type'] if \
                                                        'multi_cross_attention' in config['params']['model_params']['combine_params'] else None)
        version.append(backbone[-1]+pooling[-1])

        exp_loss = []
        for i in range(len(config['stats']['train'])):
            exp_loss.append(float(config['stats']['train'][i]['loss']))
        loss.append(exp_loss)
        
        exp_avg_recall, exp_1p_avg_recall = [], []
        for j in range(len(config['stats']['eval'])):
            epoch = re.match(r'(epoch[0-9]+)', next(iter(config['stats']['eval'][j]))).group(1)
            exp_1p_avg_recall.append(float(config['stats']['eval'][j][epoch][config['params']['dataset_name'].lower()]['ave_one_percent_recall']))
            exp_avg_recall_str = config['stats']['eval'][j][epoch][config['params']['dataset_name'].lower()]['ave_recall']
            exp_avg_recall.append(float(re.match(r'\[\s*([0-9]+.[0-9]+).*', exp_avg_recall_str).group(1)))
        average_recall.append(exp_avg_recall)
        average_1p_recall.append(exp_1p_avg_recall)
        assert len(average_recall) == len(average_1p_recall), 'The length of average_recall and average_1p_recall is not equal.'
    
    model_names = []
    for v, p, s, pc, m in zip(version, pointnet, self_attention, pointnet_cross_attention, multi_cross_attention):
        model_name = v
        model_name = model_name + ' + pointnet' if p else model_name
        model_name = model_name + ' + self_attention' if s else model_name
        model_name = model_name + ' + pointnet_cross_attention' if pc else model_name
        model_name = model_name + ' + multi_cross_attention' if m else model_name
        if v == 'MinkFPNGeM' and not p and not s and not pc and not m:
            model_name = model_name + ' Baseline'
        model_names.append(model_name)

    dirs = [d.replace('-', '_')[:-2] if int(d.split('-')[0][-3:]) < 731 else d.replace('-', '_') for d in dirs]
    dirs = [f'model_{v}_{d}' for v,d in zip(version, dirs)]

    df = pd.DataFrame(
        {   'dataset_name': datasets,
            'model_name': model_names,
            'dir': dirs,
            'epochs': epochs,
            'num_points': num_points,
            'batch_size': batch_size,
            'batch_size_limit': batch_size_limit,
            'version': version,
            'pointnet.pnt2s': pointnet_pnt2s,
            'self_attention.num_layers': self_attention_num_layers,
            'pointnet_cross_attention.attention_types': pointnet_cross_attention_attention_types,
            'loss': loss,
            'multi_cross_attention': multi_cross_attention,
        })

    metrics = {
        'average_recall': np.array(average_recall),
        'average_1p_recall': np.array(average_1p_recall)
    }
    return df, metrics


def get_highest_by_criterion(criterion, df, metrics):
    all_metrics = ['average_recall', 'average_1p_recall']
    assert criterion in all_metrics, 'criterion must be one of {}'.format(all_metrics)
    
    criterion1 = next(iter (set(all_metrics) - set([criterion])))
    max_vals = [np.max(l) for l in metrics[criterion]]
    max_idx =  [np.argmax(l) for l in metrics[criterion]]
    assert len(max_vals) == len(max_idx), 'The length of max_vals and max_idx is not equal.'
    df['best_epoch'] = np.asarray(max_idx) + 1
    df.loc[df['dataset_name'] != 'TUM', 'best_epoch'] =  df.loc[df['epochs'] != 'TUM', 'epochs']
    df[criterion] = np.asarray(max_vals, dtype="object")
    df[criterion1] = np.asarray([l[i] for i, l in zip(max_idx, metrics[criterion1])],  dtype="object")
    assert criterion in df.columns and criterion1 in df.columns and 'best_epoch' in df.columns
    
    return df

def get_results(tf_logdir):
    """
    Get the table of results, return a dataframe.
    """
    if not os.path.exists(tf_logdir):
        raise ValueError('The directory {} does not exist.'.format(tf_logdir))

    configs, dirs = get_finished_configs(tf_logdir)

    df = get_columns(configs, dirs)

    return df 

In [3]:
df, metrics = get_results(tf_logdir)

/tmp/ipykernel_46943/529026454.py:116: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  'average_recall': np.array(average_recall),
/tmp/ipykernel_46943/529026454.py:117: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  'average_1p_recall': np.array(average_1p_recall)


In [4]:
pd.set_option('display.max_rows',None)
criterion = 'average_recall'
assert criterion in ['average_1p_recall', 'average_recall']
df_criterion = get_highest_by_criterion(criterion, df, metrics)
df_sort_avg_recall = df_criterion.sort_values(by=['dataset_name', criterion, 'batch_size', 'batch_size_limit'], ascending=False)
df_sort_avg_recall

,dataset_name,model_name,dir,epochs,num_points,batch_size,batch_size_limit,version,pointnet.pnt2s,self_attention.num_layers,pointnet_cross_attention.attention_types,loss,multi_cross_attention,best_epoch,average_recall,average_1p_recall
54,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_165246,40,23000,32,84,MinkFPNGeM,None,4.0,None,"[0.297370051061828, 0.21859290280094829, 0.202...",False,40,91.267665,99.339217
7,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_234137,40,23000,32,84,MinkFPNGeM,None,4.0,None,"[0.2932179558112669, 0.21791595787755236, 0.19...",False,40,91.035584,99.359674
66,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_164920,40,23000,32,84,MinkFPNGeM,None,3.0,None,"[0.31661331836744694, 0.21766271042304303, 0.1...",False,40,91.016165,99.191143
25,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_234258,40,23000,32,84,MinkFPNGeM,None,5.0,None,"[0.2916132943536526, 0.21590004753141054, 0.19...",False,40,91.00476,99.211791
49,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_163231,40,23000,32,84,MinkFPNGeM,None,2.0,None,"[0.31368727876141317, 0.22083832256906077, 0.2...",False,40,90.675007,99.17917
57,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220808_134106,40,23000,32,84,MinkFPNGeM,None,6.0,None,"[0.2143100021425697, 0.20844471995689237, 0.20...",False,40,90.170299,99.297374
42,USyd,MinkFPNGeM Baseline,model_MinkFPNGeM_20220806_235236,40,23000,32,84,MinkFPNGeM,None,NaN,None,"[0.32053131185297584, 0.22903381810959325, 0.2...",False,40,89.902559,99.228889
14,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220730_1221,40,23000,32,84,MinkFPNGeM,None,1.0,None,"[0.31484471999083874, 0.2260664991111657, 0.20...",False,40,89.818177,99.169297
29,USyd,MinkFPNGeM + self_attention + pointnet_cross_a...,model_MinkFPNGeM_20220818_141758,40,4096,10,10,MinkFPNGeM,None,3.0,dot_prod,"[0.28179509775913664, 0.19499375960587795, 0.1...",False,40,87.758812,98.705604
73,USyd,MinkFPNGeM + self_attention + pointnet_cross_a...,model_MinkFPNGeM_20220814_122925,40,4096,10,10,MinkFPNGeM,None,3.0,dot_prod,"[0.2906843660184135, 0.20235380516253781, 0.18...",False,40,86.832914,98.756944


In [5]:
criterion = 'average_1p_recall'
assert criterion in ['average_1p_recall', 'average_recall']
df_criterion = get_highest_by_criterion(criterion, df, metrics)
df_sort_avg_1p_recall = df_criterion.sort_values(by=['dataset_name', criterion, 'batch_size', 'batch_size_limit'], ascending=False)
df_sort_avg_1p_recall

,dataset_name,model_name,dir,epochs,num_points,batch_size,batch_size_limit,version,pointnet.pnt2s,self_attention.num_layers,pointnet_cross_attention.attention_types,loss,multi_cross_attention,best_epoch,average_recall,average_1p_recall
7,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_234137,40,23000,32,84,MinkFPNGeM,None,4.0,None,"[0.2932179558112669, 0.21791595787755236, 0.19...",False,40,91.035584,99.359674
54,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_165246,40,23000,32,84,MinkFPNGeM,None,4.0,None,"[0.297370051061828, 0.21859290280094829, 0.202...",False,40,91.267665,99.339217
57,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220808_134106,40,23000,32,84,MinkFPNGeM,None,6.0,None,"[0.2143100021425697, 0.20844471995689237, 0.20...",False,40,90.170299,99.297374
42,USyd,MinkFPNGeM Baseline,model_MinkFPNGeM_20220806_235236,40,23000,32,84,MinkFPNGeM,None,NaN,None,"[0.32053131185297584, 0.22903381810959325, 0.2...",False,40,89.902559,99.228889
25,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_234258,40,23000,32,84,MinkFPNGeM,None,5.0,None,"[0.2916132943536526, 0.21590004753141054, 0.19...",False,40,91.00476,99.211791
66,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_164920,40,23000,32,84,MinkFPNGeM,None,3.0,None,"[0.31661331836744694, 0.21766271042304303, 0.1...",False,40,91.016165,99.191143
49,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220806_163231,40,23000,32,84,MinkFPNGeM,None,2.0,None,"[0.31368727876141317, 0.22083832256906077, 0.2...",False,40,90.675007,99.17917
14,USyd,MinkFPNGeM + self_attention,model_MinkFPNGeM_20220730_1221,40,23000,32,84,MinkFPNGeM,None,1.0,None,"[0.31484471999083874, 0.2260664991111657, 0.20...",False,40,89.818177,99.169297
73,USyd,MinkFPNGeM + self_attention + pointnet_cross_a...,model_MinkFPNGeM_20220814_122925,40,4096,10,10,MinkFPNGeM,None,3.0,dot_prod,"[0.2906843660184135, 0.20235380516253781, 0.18...",False,40,86.832914,98.756944
29,USyd,MinkFPNGeM + self_attention + pointnet_cross_a...,model_MinkFPNGeM_20220818_141758,40,4096,10,10,MinkFPNGeM,None,3.0,dot_prod,"[0.28179509775913664, 0.19499375960587795, 0.1...",False,40,87.758812,98.705604


### Best Avg Recall

In [11]:
best_tum_avg_recall_model_desc = df_sort_avg_recall.loc[df_sort_avg_recall['dataset_name'] == 'TUM'].iloc[0]
best_tum_avg_recall_dir, best_tum_avg_recall_best_epoch = best_tum_avg_recall_model_desc['dir'], best_tum_avg_recall_model_desc['best_epoch']
best_tum_avg_recall_model = os.path.join(weights_dir, best_tum_avg_recall_dir, f'epoch{best_tum_avg_recall_best_epoch}.pth')
assert os.path.exists(best_tum_avg_recall_model), f'{best_tum_avg_recall_model} not exists'
print('Best TUM average recall model:\n\n', best_tum_avg_recall_model_desc)

Best TUM average recall model:

 dataset_name                                                                              TUM
model_name                                                        MinkFPNGeM + self_attention
dir                                                            model_MinkFPNGeM_20220730_2352
epochs                                                                                    150
num_points                                                                               4096
batch_size                                                                                 16
batch_size_limit                                                                           32
version                                                                            MinkFPNGeM
pointnet.pnt2s                                                                           None
self_attention.num_layers                                                                 2.0
pointnet_cross_attention.at

In [12]:
best_usyd_avg_recall_model_desc = df_sort_avg_recall.loc[df_sort_avg_recall['dataset_name'] == 'USyd'].iloc[0]
best_usyd_avg_recall_dir, best_usyd_avg_recall_best_epoch = best_usyd_avg_recall_model_desc['dir'], best_usyd_avg_recall_model_desc['best_epoch']
best_usyd_avg_recall_model = os.path.join(weights_dir, best_usyd_avg_recall_dir, f'epoch{best_usyd_avg_recall_best_epoch}.pth')
assert os.path.exists(best_usyd_avg_recall_model), f'{best_usyd_avg_recall_model} not exists'
print('Best USyd average recall model:\n\n', best_usyd_avg_recall_model_desc)

Best USyd average recall model:

 dataset_name                                                                             USyd
model_name                                                        MinkFPNGeM + self_attention
dir                                                          model_MinkFPNGeM_20220806_165246
epochs                                                                                     40
num_points                                                                              23000
batch_size                                                                                 32
batch_size_limit                                                                           84
version                                                                            MinkFPNGeM
pointnet.pnt2s                                                                           None
self_attention.num_layers                                                                 4.0
pointnet_cross_attention.a

### Best Avg 1p Recall

In [13]:
best_tum_avg_1p_recall_model_desc = df_sort_avg_1p_recall.loc[df_sort_avg_1p_recall['dataset_name'] == 'TUM'].iloc[0]
best_tum_avg_1p_recall_dir, best_tum_avg_1p_recall_best_epoch = best_tum_avg_1p_recall_model_desc['dir'], best_tum_avg_1p_recall_model_desc['best_epoch']
best_tum_avg_1p_recall_model = os.path.join(weights_dir, best_tum_avg_1p_recall_dir, f'epoch{best_tum_avg_1p_recall_best_epoch}.pth')
assert os.path.exists(best_tum_avg_1p_recall_model), f'{best_tum_avg_1p_recall_model} not exists'
print('Best TUM average 1p recall model:\n\n', best_tum_avg_1p_recall_model_desc)

Best TUM average 1p recall model:

 dataset_name                                                                              TUM
model_name                                  MinkFPNGeM + self_attention + pointnet_cross_a...
dir                                                          model_MinkFPNGeM_20220813_115522
epochs                                                                                    150
num_points                                                                               4096
batch_size                                                                                 10
batch_size_limit                                                                           10
version                                                                            MinkFPNGeM
pointnet.pnt2s                                                                           None
self_attention.num_layers                                                                 5.0
pointnet_cross_attention

In [14]:
best_usyd_avg_1p_recall_model_desc = df_sort_avg_1p_recall.loc[df_sort_avg_1p_recall['dataset_name'] == 'USyd'].iloc[0]
best_usyd_avg_1p_recall_dir, best_usyd_avg_1p_recall_best_epoch = best_usyd_avg_1p_recall_model_desc['dir'], best_usyd_avg_1p_recall_model_desc['best_epoch']
best_usyd_avg_1p_recall_model = os.path.join(weights_dir, best_usyd_avg_1p_recall_dir, f'epoch{best_usyd_avg_1p_recall_best_epoch}.pth')
assert os.path.exists(best_usyd_avg_1p_recall_model), f'{best_usyd_avg_1p_recall_model} not exists'
print('Best USyd average 1p recall model:\n\n', best_usyd_avg_1p_recall_model_desc)

Best USyd average 1p recall model:

 dataset_name                                                                             USyd
model_name                                                        MinkFPNGeM + self_attention
dir                                                          model_MinkFPNGeM_20220806_234137
epochs                                                                                     40
num_points                                                                              23000
batch_size                                                                                 32
batch_size_limit                                                                           84
version                                                                            MinkFPNGeM
pointnet.pnt2s                                                                           None
self_attention.num_layers                                                                 4.0
pointnet_cross_attentio

In [15]:
best_models = [best_tum_avg_recall_model,
               best_usyd_avg_recall_model,
               best_tum_avg_1p_recall_model,
               best_usyd_avg_1p_recall_model]

In [17]:
best_models

['../weights/model_MinkFPNGeM_20220730_2352/epoch56.pth',
 '../weights/model_MinkFPNGeM_20220806_165246/epoch40.pth',
 '../weights/model_MinkFPNGeM_20220813_115522/epoch114.pth',
 '../weights/model_MinkFPNGeM_20220806_234137/epoch40.pth']

# Run Experiment

## Usyd

In [11]:
# %cd /home/xiayan/testdir/MinkLoc3D-SI/training
# ! nohup python train.py \
#     --config=../config_usyd.txt \
#     --model_config=../config/model_config.txt > ${log_file_name}.log  2>&1 & 

## TUM

In [12]:
# %cd /home/xiayan/testdir/MinkLoc3D-SI/training
# ! nohup python train.py \
#     --config=../config_tum.txt \
#     --model_config=../config/model_config.txt > ${log_file_name}.log  2>&1 & 

## Evaluate Kitti

In [13]:
default_usyd_model = os.path.join(weights_dir, 'MinkLoc3D-SI-USyd.pth')

### NOTE
  
* Change model_config.txt into the configuration of the stored model before running (using the printed information of the best models above)
* Advice: 
    * check the config of each best model and save the txt files into a separate folder called 'best_models'
    * change the weights paths into 'best_models/best_xx_model.txt' so that you don need to change 'model_config.txt' each time

### Default Usyd

In [14]:
%cd /home/xiayan/testdir/MinkLoc3D-SI/eval

/home/xiayan/testdir/MinkLoc3D-SI/eval


In [18]:
! python evaluate_kitti.py --config=../config/config_kitti.txt \
     --model_config=../config/model_config.txt \
     --weights=$default_usyd_model

/root/miniconda3/envs/mink/lib/python3.8/site-packages/MinkowskiEngine-0.5.4-py3.8-linux-x86_64.egg/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


### Best Usyd Avg Recall

In [ ]:
# %cd /home/xiayan/testdir/MinkLoc3D-SI/eval
# ! python evaluate_kitti.py --config=../config/config_kitti.txt \
#      --model_config=../config/model_config.txt \
#      --weights=$best_usyd_avg_recall_model

### Best Avg 1p Recall Usyd

In [ ]:
# %cd /home/xiayan/testdir/MinkLoc3D-SI/eval
# ! python evaluate_kitti.py --config=../config/config_kitti.txt \
#      --model_config=../config/model_config.txt \
#      --weights=$best_usyd_avg_1p_recall_model

### Best TUM Avg Recall

In [ ]:
# %cd /home/xiayan/testdir/MinkLoc3D-SI/eval
# ! python evaluate_kitti.py --config=../config/config_kitti.txt \
#      --model_config=../config/model_config.txt \
#      --weights=$best_tum_avg_recall_model

### Best TUM Avg 1p Recall

In [ ]:
%cd /home/xiayan/testdir/MinkLoc3D-SI/eval
! python evaluate_kitti.py --config=../config/config_kitti.txt \
     --model_config=../config/model_config.txt \
     --weights=$best_tum_avg_1p_recall_model